In [1]:
def zero_matrix(nrow, ncol):
    M = []
    while len(M) < nrow:
        M.append([]) # create a new row
        while len(M[-1]) < ncol:
            M[-1].append(0.0)
    return M

In [2]:
def zero_array_3d(time, row, col):
    M = zero_matrix(row, col)
    M_3d = [M]
    while len(M_3d) < time:
        M_3d.append(M)
    return M_3d

In [3]:
# HMM2 - Decoding
from math import log

In [4]:
with open('hmm4_01.in') as inp:
    A_list = list(map(float, inp.readline().split()))
    B_list = list(map(float, inp.readline().split()))
    pi_list = list(map(float, inp.readline().split()))
    obs_list = list(map(int, inp.readline().split()))
    print(A_list, type(A_list))
    print(B_list)
    print(pi_list)
    print(obs_list[0])

[4.0, 4.0, 0.4, 0.2, 0.2, 0.2, 0.2, 0.4, 0.2, 0.2, 0.2, 0.2, 0.4, 0.2, 0.2, 0.2, 0.2, 0.4] <class 'list'>
[4.0, 4.0, 0.4, 0.2, 0.2, 0.2, 0.2, 0.4, 0.2, 0.2, 0.2, 0.2, 0.4, 0.2, 0.2, 0.2, 0.2, 0.4]
[1.0, 4.0, 0.241896, 0.266086, 0.249153, 0.242864]
1000


In [5]:
with open('hmm4_01.ans') as ans:
    ans_list = ans.readline().split()
print(ans_list)

['4', '4', '0.545455', '0.454545', '0.0', '0.0', '0.0', '0.506173', '0.493827', '0.0', '0.0', '0.0', '0.504132', '0.495868', '0.478088', '0.0', '0.0', '0.521912']


In [6]:
N = int(A_list[0])
K = int(B_list[1])
T = obs_list[0]
print(T, type(T))
print(K)

1000 <class 'int'>
4


In [7]:
# initial lambda

A = zero_matrix(N, N)
B = zero_matrix(N, K)
pi = pi_list[2:]
# pi = [pi_list[2: ]]

obs = obs_list[1:]

In [8]:
print(len(pi))

4


In [9]:
for i in range(N):
    for j in range(N):
        A[i][j] = A_list[2 + N*i + j]
    for j in range(K):
        B[i][j] = B_list[2 + K*i + j]

In [10]:
print(B)
print(A)

[[0.4, 0.2, 0.2, 0.2], [0.2, 0.4, 0.2, 0.2], [0.2, 0.2, 0.4, 0.2], [0.2, 0.2, 0.2, 0.4]]
[[0.4, 0.2, 0.2, 0.2], [0.2, 0.4, 0.2, 0.2], [0.2, 0.2, 0.4, 0.2], [0.2, 0.2, 0.2, 0.4]]


In [11]:
# alpha-pass: ouput alphas as a T * N matrix, and coefficients c_t
def alpha_pass_scaled(A, B, pi, obs):
    # by log
    T = len(obs)
    N = len(A)
    alphas = zero_matrix(T, N)
    c = [0.0] * T
    alpha_tilde = [pi[i] * B[i][obs[0]] for i in range(N)]
    c[0] = 1.0 / sum(alpha_tilde)
    alphas[0] = [c[0] * alpha for alpha in alpha_tilde]
    for t in range(1, T):
        alpha_tilde = [sum(alphas[t-1][j] * A[j][i] * B[i][obs[t]] for j in range(N)) for i in range(N)]
        c[t] = 1.0 / sum(alpha_tilde)
        alphas[t] = [c[t] * alpha for alpha in alpha_tilde]
    return alphas, c

In [12]:
alphas, c = alpha_pass_scaled(A, B, pi, obs)
print(alphas[T-1])

[0.43334919514301057, 0.18667402956642004, 0.18668889861131918, 0.19328787667925015]


In [13]:
# beta-pass
def beta_pass_scaled(A, B, obs, c):
    T = len(obs)
    N = len(A)
    betas = zero_matrix(T, N)
    betas[T-1] = [c[T-1]] * N
    for t in range(T-2, -1, -1):
        betas[t] = [sum(betas[t+1][j] * B[j][obs[t+1]] * A[i][j] for j in range(N)) * c[t] for i in range(N) ]
    return betas

In [14]:
betas = beta_pass_scaled(A, B, obs, c)
print(betas[T-1])

[3.916627012142473, 3.916627012142473, 3.916627012142473, 3.916627012142473]


In [38]:
def gamma_func(A, B, pi, obs):
    T = len(obs)
    N = len(A)
    alphas, c = alpha_pass_scaled(A, B, pi, obs)
    betas = beta_pass_scaled(A, B, obs, c)
    di_gammas = zero_array_3d(T-1, N, N)
    # gammas = zero_matrix(T-1, N)
    gammas = zero_matrix(T, N)
    gammas[T-1] = [alphas[T-1][i] for i in range(N)]
    for t in range(T-1):
        for i in range(N):
            gammas[t][i] = 0
            for j in range(N):
                di_gammas[t][i][j] = alphas[t][i] * A[i][j] * B[j][obs[t+1]] * betas[t+1][j] # / sum(alphas[T-1][k] for k in range(N))
                gammas[t][i] += di_gammas[t][i][j]
    return gammas, di_gammas, c

In [39]:
gammas, di_gammas, c = gamma_func(A, B, pi, obs)
# print(gammas[T-1])

In [40]:
def baum_welch(A, B, pi, obs):
    # given a starting guess
    N = len(A)
    K = len(B[0])
    T = len(obs)
    # Compute di-gamma and gamma functions
    gammas, di_gammas, c = gamma_func(A, B, pi, obs)
    new_A = zero_matrix(N, N)
    new_B = zero_matrix(N, K)
    new_pi = None
    # re-estimate lambda
    for i in range(N):
        denom = 0
        for t in range(T-1):
            denom += gammas[t][i]
        for j in range(N):
            numr = 0
            for t in range(T-1):
                numr += di_gammas[t][i][j]
            new_A[i][j] = numr / denom
    for i in range(N):
        denom = 0
        for t in range(T):
            denom += gammas[t][i]
        for k in range(K):
            numr = 0
            for t in range(T):
                numr += (obs[t] == k) * gammas[t][i]
            new_B[i][k] = numr / denom
    new_pi = gammas[0]
    return new_A, new_B, new_pi, c

In [41]:
def learning(A, B, pi, obs):
    maxIters = 500
    loss = 1e-3
    oldLogProb = 0
    for iters in range(maxIters):
        A, B, pi, c = baum_welch(A, B, pi, obs)
        # Compute log P(O|lambda)
        logProb = -sum(log(ct) for ct in c)
        if iters != 0 and (logProb <= oldLogProb or abs(logProb - oldLogProb) <= loss):
            break
        oldLogProb = logProb
    return A, B

In [42]:
A, B = learning(A, B, pi, obs)
print(A)
print(B)

[[0.9995321044031753, 1.1309901000777751e-18, 1.9073918675513294e-11, 0.0003475666721322846], [1.864632742785834e-17, 8.439473481967727e-35, 7.116500438339258e-28, 1.2967751497006907e-20], [9.302746874884817e-10, 2.1052479600553888e-27, 7.100915981338373e-20, 6.469676679215751e-13], [3.808951066538222, 8.619805064706163e-18, 1.453712643388241e-10, 0.005297936667284928]]
[[0.26396830573177826, 0.24301046427605777, 0.24201042121319294, 0.25101080877897264], [1.0, 2.590629336756601e-88, 1.2781746475938242e-88, 5.8097892495383994e-89], [1.0, 3.378312700770176e-52, 1.1691188672701373e-51, 3.0703304411182097e-52], [1.0, 3.3820547663903045e-23, 5.592961958875976e-23, 9.899762667432217e-23]]
